
## Importing Libraries


In [47]:
import os
from dotenv import load_dotenv
from graphdatascience import GraphDataScience
import pandas as pd

## Load and retreive neo4j variables from .env file

In [48]:
load_dotenv()

#Storing Neo4j connection details in variables
URI = os.getenv("NEO4J_URI")
USER = os.getenv("NEO4J_USERNAME")
PASSWORD = os.getenv("NEO4J_PASSWORD")
DB_NAME = os.getenv("NEO4J_DATABASE", "neo4j")

## Initialize the Graph Data Science client

In [49]:
gds = GraphDataScience(
    URI,
    auth=(USER, PASSWORD),
    database=DB_NAME
)

In [50]:

print("Connected to Neo4j GDS server version:", gds.version())

Connected to Neo4j GDS server version: 2.23.0


In [51]:
print("Available GDS algorithms (first few rows):")
print(gds.list().head())

Available GDS algorithms (first few rows):
                                         name  \
0           gds.allShortestPaths.delta.mutate   
1  gds.allShortestPaths.delta.mutate.estimate   
2            gds.allShortestPaths.delta.stats   
3   gds.allShortestPaths.delta.stats.estimate   
4           gds.allShortestPaths.delta.stream   

                                         description  \
0  The Delta Stepping shortest path algorithm com...   
1  Returns an estimation of the memory consumptio...   
2  The Delta Stepping shortest path algorithm com...   
3  Returns an estimation of the memory consumptio...   
4  The Delta Stepping shortest path algorithm com...   

                                           signature       type  
0  gds.allShortestPaths.delta.mutate(graphName ::...  procedure  
1  gds.allShortestPaths.delta.mutate.estimate(gra...  procedure  
2  gds.allShortestPaths.delta.stats(graphName :: ...  procedure  
3  gds.allShortestPaths.delta.stats.estimate(grap...  procedu

## Graph Projections


In [52]:
# Cleanup existing graph projection if it exists
if gds.graph.exists("foodWebGraph").exists:
    gds.graph.drop("foodWebGraph")

In [53]:
# First projection for directed analyses (degree, betweenness, closeness, communities)
G, proj_result = gds.graph.project(
    "foodWebGraph",
    {
        "Taxa": {
            "properties": [
                "taxon_id"
            ]
        }
    },
    {
        "eaten_by": {
            "orientation": "NATURAL",   # direction stays Predator -> Prey
            "properties": [
                "latitude",
                "longitude"
                ]
        }
    }
)


In [54]:
print(f"Projected {G.node_count()} nodes and {G.relationship_count()} relationships for directed analyses.\n")

Projected 3320 nodes and 2890 relationships for directed analyses.



## Data Correction


In [55]:
# Update the Scientific Name of the some null values

# cypher = """
# MATCH (n:Taxa {taxon_id: $id})
# SET n.scientific_name = "Xysmalobium undulatum"
# RETURN n.taxon_id AS taxon_id, n.habitat AS habitat
# """
# result = gds.run_cypher(cypher, params={"id": 123, "value": "estuary"})
# print(result)


In [56]:
props_df = gds.run_cypher("""
MATCH ()-[r:eaten_by]->()
WITH r LIMIT 1000              // adjust if needed
UNWIND keys(r) AS k
RETURN k AS property, count(*) AS freq
ORDER BY freq DESC
""")
print(props_df)

                    property  freq
0                       uuid  1000
1   predator_scientific_name  1000
2       prey_scientific_name  1000
3                observed_on  1000
4                        url  1000
5                  image_url  1000
6           predator_quality  1000
7               prey_quality  1000
8                   latitude   996
9                  longitude   996
10                     place   994
11               place_state   993
12             place_country   993
13              place_county   987
14          time_observed_at   966
15           special_feeding   694
16                place_town    73


In [57]:
query = """
MATCH (s:Taxa)-[r:eaten_by]->(t:Taxa)
RETURN t.scientific_name AS predator_scientific_name,
       t.taxon_kingdom   AS predator_taxon_kingdom,
       s.scientific_name AS prey_scientific_name,
       s.taxon_kingdom   AS prey_taxon_kingdom
"""
df = gds.run_cypher(query)
df.to_csv("relationships.csv", index=False)

### Degree centrality

In [58]:
# Degree centrality
degree_centrality = gds.degree.write(
    G, 
    writeProperty="degree"
)

print("Degree centrality computed and written to the graph as 'degree' property.\n")

Degree centrality computed and written to the graph as 'degree' property.



### Betweenness centrality

In [59]:
# Betweenness centrality
betweenness_centrality = gds.betweenness.write(
    G, 
    writeProperty="betweenness"
)
print("Betweenness centrality computed and written to the graph as 'betweenness' property.\n")


Betweenness centrality computed and written to the graph as 'betweenness' property.



### Closeness centrality

In [60]:
# Closeness centrality
closeness_centrality = gds.closeness.write(
    G, 
    writeProperty="closeness"
)
print("Closeness centrality computed and written to the graph as 'closeness' property.\n")


Closeness centrality computed and written to the graph as 'closeness' property.



### Detecting communities using louvian method:

What is Louvian Method for reference:

In [61]:
# Using Louvain method for community detection
communities = gds.louvain.write(
    G,
    writeProperty="community"
)
print("Communities detected using Louvain method and written to the graph as 'community' property.\n")

 Louvain: 100%|██████████| 100.0/100 [00:01<00:00, 81.21%/s, status: FINISHED] 

Communities detected using Louvain method and written to the graph as 'community' property.



In [62]:
#Identify cyclic regions (SCCs)
cycles = gds.scc.write(G, writeProperty='sccId')

## Retriving the calculated metrics

### Statistic Summary

In [63]:
# Statistic summary
stats_query = """
MATCH (n:Taxa)
RETURN 
    'Degree' as metric,
    min(n.degree) as min,
    max(n.degree) as max,
    avg(n.degree) as avg,
    count(n) as count
UNION
MATCH (n:Taxa)
RETURN 
    'Betweenness' as metric,
    min(n.betweenness) as min,
    max(n.betweenness) as max,
    avg(n.betweenness) as avg,
    count(n) as count
UNION
MATCH (n:Taxa)
RETURN 
    'Closeness' as metric,
    min(n.closeness) as min,
    max(n.closeness) as max,
    avg(n.closeness) as avg,
    count(n) as count
"""

In [64]:
# Run the query and get results as a pandas DataFrame
stats_df = gds.run_cypher(stats_query)

print("\nMetric Statistics:")
print(stats_df)


Metric Statistics:
        metric  min    max       avg  count
0       Degree  0.0   38.0  0.870482   3320
1  Betweenness  0.0  649.5  0.688554   3320
2    Closeness  0.0    1.0  0.442194   3320


In [65]:
# Query to retrieve calculated metrics for each node
metrics_query = """
MATCH (n:Taxa)
RETURN n.taxon_id as taxon_id,
       n.common_name as common_name,
       n.scientific_name as scientific_name,
       n.degree as degree,
       n.betweenness as betweenness,
       n.closeness as closeness,
       n.community as community
ORDER BY n.betweenness DESC
LIMIT 10
"""

In [66]:
# Run the query and get results as a pandas DataFrame
metrics_df = gds.run_cypher(metrics_query)


In [67]:

print("\nTop 10 species by betweenness centrality:")
print(metrics_df)


Top 10 species by betweenness centrality:
   taxon_id                       common_name  \
0   47219.0                 Western Honey Bee   
1   61355.0              Western Yellowjacket   
2  199400.0                         Siam weed   
3  676794.0              Common Dotted Border   
4   46017.0             Eastern Gray Squirrel   
5   12727.0                    American Robin   
6   46260.0             American Red Squirrel   
7   61495.0                  Eastern Pondhawk   
8  625887.0  Southern Brown-hooded Kingfisher   
9  483731.0                              None   

                   scientific_name  degree  betweenness  closeness  community  
0                   Apis mellifera    14.0        649.5   0.976190       1515  
1             Vespula pensylvanica     3.0        132.0   0.511628        215  
2              Chromolaena odorata     5.0        130.0   0.490566       2535  
3      Mylothris agathina agathina     2.0        104.0   0.464286       2535  
4             Sci

In [68]:
print("\nTop 10 species with details:")
print(metrics_df)


Top 10 species with details:
   taxon_id                       common_name  \
0   47219.0                 Western Honey Bee   
1   61355.0              Western Yellowjacket   
2  199400.0                         Siam weed   
3  676794.0              Common Dotted Border   
4   46017.0             Eastern Gray Squirrel   
5   12727.0                    American Robin   
6   46260.0             American Red Squirrel   
7   61495.0                  Eastern Pondhawk   
8  625887.0  Southern Brown-hooded Kingfisher   
9  483731.0                              None   

                   scientific_name  degree  betweenness  closeness  community  
0                   Apis mellifera    14.0        649.5   0.976190       1515  
1             Vespula pensylvanica     3.0        132.0   0.511628        215  
2              Chromolaena odorata     5.0        130.0   0.490566       2535  
3      Mylothris agathina agathina     2.0        104.0   0.464286       2535  
4             Sciurus caroline

### Community Statistics

In [69]:

community_stats_query = """
MATCH (n:Taxa)
WITH n.community as community, count(*) as size
RETURN 
    community,
    size
ORDER BY size DESC
"""

In [70]:
# Run the query and get results as a pandas DataFrame
community_df = gds.run_cypher(community_stats_query)

In [71]:
print("\nTop largest communities:")
print(community_df)


Top largest communities:
     community  size
0         1302   303
1          215   162
2         2861   160
3         1515   152
4         2535   141
..         ...   ...
842       3245     1
843       3251     1
844       3261     1
845       3296     1
846       3310     1

[847 rows x 2 columns]


## Finding long chains using APOC path expansion

In [72]:
long_chain = """
MATCH (s:Taxa)
CALL apoc.path.expandConfig(s, {
  relationshipFilter: '<eaten_by',
  minLevel: 4,
  maxLevel: 50,
  uniqueness: 'NODE_GLOBAL',
  bfs: false,
  limit: 100000
}) YIELD path
WITH path, length(path) AS len
RETURN apoc.text.join([n IN nodes(path) | n.scientific_name], ' -> ') AS chain, len 
ORDER BY len DESC
LIMIT 50;

"""

In [80]:
long_chain_df = gds.run_cypher(long_chain)
print("\nTop 10 longest food chains found:")
print(long_chain_df)
long_chain_df.to_csv("Data/long_food_chains.csv", index=False)


Top 10 longest food chains found:
                                                chain  len
0   Eccopsis incultana -> Lippia javanica -> Mylot...    5
1   Eccopsis incultana -> Lippia javanica -> Mylot...    5
2   Eccopsis incultana -> Lippia javanica -> Mylot...    5
3   Eccopsis incultana -> Lippia javanica -> Mylot...    5
4   Eccopsis incultana -> Lippia javanica -> Mylot...    5
5   Eccopsis incultana -> Lippia javanica -> Mylot...    5
6   Eccopsis incultana -> Lippia javanica -> Mylot...    5
7   Eccopsis incultana -> Lippia javanica -> Mylot...    5
8   Eccopsis incultana -> Lippia javanica -> Mylot...    5
9   Eccopsis incultana -> Lippia javanica -> Mylot...    5
10  Pantherophis spiloides -> Strix varia -> Tamia...    5
11  Pantherophis spiloides -> Strix varia -> Tamia...    5
12  Lippia javanica -> Mylothris agathina agathina...    4
13  Lippia javanica -> Mylothris agathina agathina...    4
14  Melanerpes lewis -> Bombus griseocollis -> Asc...    4
15  Neoscona oaxacens

In [74]:
display_cycles = """
MATCH (n:Taxa)
WITH n.sccId AS scc, count(*) AS size
WHERE size > 1
RETURN scc, size
ORDER BY size DESC
LIMIT 20;
"""

In [75]:
length_cycles_df = gds.run_cypher(display_cycles)
print("\nTop 20 cycles:")
print(length_cycles_df)


Top 20 cycles:
     scc  size
0   3002     6
1    173     3
2     11     2
3    243     2
4    333     2
5    381     2
6    443     2
7   1306     2
8   1576     2
9   1703     2
10  2011     2


In [76]:
cyclic_chains="""
MATCH (start:Taxa {sccId: $sccId})
CALL apoc.path.expandConfig(start, {
  relationshipFilter: '<eaten_by',
  minLevel: 2,
  maxLevel: 12,
  filterStartNode: true,
  where: 'node.sccId = startNode.sccId',
  terminatorNodes: [start],
  limit: 100000
}) YIELD path
WHERE last(nodes(path)) = start
WITH path, length(path) AS len
RETURN [n IN nodes(path) | n.scientific_name] AS scientific_names, len
ORDER BY len DESC
LIMIT 50;
"""

In [77]:
cyclic_chains_df = gds.run_cypher(cyclic_chains,{"sccId": 3002})
print("\nTop 20 cycles:")
print(cyclic_chains_df)



Top 20 cycles:
                                     scientific_names  len
0   [Belenois creona severina, Chromolaena odorata...   10
1   [Belenois creona severina, Chromolaena odorata...   10
2   [Belenois creona severina, Chromolaena odorata...   10
3   [Belenois creona severina, Chromolaena odorata...   10
4   [Belenois creona severina, Chromolaena odorata...   10
5   [Belenois creona severina, Chromolaena odorata...   10
6   [Delta fenestrale, Chromolaena odorata, Beleno...   10
7   [Delta fenestrale, Chromolaena odorata, Beleno...   10
8   [Delta fenestrale, Chromolaena odorata, Myloth...   10
9   [Delta fenestrale, Chromolaena odorata, Myloth...   10
10  [Delta fenestrale, Chromolaena odorata, Telchi...   10
11  [Delta fenestrale, Chromolaena odorata, Telchi...   10
12  [Lippia javanica, Mylothris agathina agathina,...   10
13  [Lippia javanica, Mylothris agathina agathina,...   10
14  [Lippia javanica, Mylothris agathina agathina,...   10
15  [Lippia javanica, Mylothris agathina

In [78]:
output_path = "Data/cyclic_chains.xlsx"  # adjust folder/name if you like
cyclic_chains_df.to_excel(output_path, index=False)

print(f"Saved {len(cyclic_chains_df)} cycles to {output_path}")

 Louvain:  15%|█▍        | 14.55/100 [08:04<47:24, 33.29s/%, status: RUNNING, task: ModularityOptimization::compute modularity::optimizeForColor]


PermissionError: [Errno 13] Permission denied: 'Data/cyclic_chains.xlsx'

### Finding the apex predators (nodes with zero in-degree)
prey_count - Count how many species this apex predator eats

In [ ]:

apex_predators_query = """
MATCH (apex:Taxa)
WHERE NOT (apex)-[:eaten_by]->()
RETURN 
    apex.taxon_id as taxon_id,
    apex.common_name as common_name,
    apex.scientific_name as scientific_name,
    apex.degree as degree,
    apex.betweenness as betweenness,
    apex.closeness as closeness,
    size([(apex)<-[:eaten_by]-(prey) | prey]) as prey_count  
ORDER BY prey_count DESC, apex.betweenness DESC
LIMIT 20
"""

In [ ]:
apex_df = gds.run_cypher(apex_predators_query)
print("\n" + "="*80)
print("APEX PREDATORS (Top of Food Chain)")
print("="*80)
print(apex_df)


APEX PREDATORS (Top of Food Chain)
     taxon_id                  common_name                   scientific_name  \
0      4956.0             Great Blue Heron                    Ardea herodias   
1     81839.0           Tan Jumping Spider              Platycryptus undatus   
2    243970.0                 Olympic Gull  Larus glaucescens × occidentalis   
3      4399.0         Glaucous-winged Gull                 Larus glaucescens   
4     55746.0        Goldenrod Crab Spider                    Misumena vatia   
5     53809.0          Bold Jumping Spider                   Phidippus audax   
6      4954.0                   Grey Heron                     Ardea cinerea   
7     12942.0             Eastern Bluebird                     Sialia sialis   
8      5305.0                   Bald Eagle          Haliaeetus leucocephalus   
9    144455.0                  Great Egret                        Ardea alba   
10    68901.0         Zebra Jumping Spider                 Salticus scenicus   
11  

In [89]:
top_predators = gds.run_cypher("""
MATCH (prey:Taxa)-[:eaten_by]->(pred:Taxa)
WITH pred, count(DISTINCT prey) AS preyCount
ORDER BY preyCount DESC
RETURN pred.taxon_id        AS taxon_id,
       pred.common_name     AS common_name,
       pred.scientific_name AS scientific_name,
       preyCount
LIMIT 5
""")
print(top_predators)
top_predators.to_csv("Data/top_predators.csv", index=False)




   taxon_id                       common_name  \
0    4956.0                  Great Blue Heron   
1   47219.0                 Western Honey Bee   
2   81839.0                Tan Jumping Spider   
3  625887.0  Southern Brown-hooded Kingfisher   
4  116999.0                            Osprey   

                   scientific_name  preyCount  
0                   Ardea herodias         47  
1                   Apis mellifera         41  
2             Platycryptus undatus         20  
3  Halcyon albiventris albiventris         20  
4                Pandion haliaetus         18  


### Finding the bottom prey(With no incoming edge)
predator_count - Count how many predators eat this prey

In [ ]:
#Finding the bottom feeders (nodes with zero out-degree)
bottom_prey_query = """
MATCH (prey:Taxa)
WHERE NOT (prey)<-[:eaten_by]-()
RETURN 
    prey.taxon_id as taxon_id,
    prey.common_name as common_name,
    prey.scientific_name as scientific_name,
    prey.degree as degree,
    size([(prey)-[:eaten_by]->(predator) | predator]) as predator_count
ORDER BY predator_count DESC
LIMIT 20
"""

In [ ]:
bottom_prey_df = gds.run_cypher(bottom_prey_query)
print("\n" + "="*80)
print("BOTTOM PREY (Base of Food Chain)")
print("="*80)
print(bottom_prey_df)


BOTTOM PREY (Base of Food Chain)
    taxon_id                              common_name  \
0    47157.0                    Butterflies and Moths   
1   585956.0                          Common Wild Fig   
2    47178.0                        Ray-finned Fishes   
3   184884.0           Winged and Once-winged Insects   
4   205898.0                          Yellow Justicia   
5    47118.0                                  Spiders   
6    47822.0                                    Flies   
7    83435.0                       American persimmon   
8    47124.0                                   dicots   
9    47158.0                                  Insects   
10  600542.0                        Coastal Resintree   
11   47649.0                Short-horned Grasshoppers   
12   47604.0  sunflowers, daisies, asters, and allies   
13   60132.0                         creeping thistle   
14  160656.0                         Spiny Fiddlewood   
15  559524.0                             Duiker-berry 

In [90]:
most_eaten_prey = gds.run_cypher("""
MATCH (prey:Taxa)-[:eaten_by]->(pred:Taxa)
WITH prey, count(DISTINCT pred) AS predatorCount
ORDER BY predatorCount DESC
RETURN prey.taxon_id        AS taxon_id,
       prey.common_name     AS common_name,
       prey.scientific_name AS scientific_name,
       predatorCount
LIMIT 5
""")
print(most_eaten_prey)
most_eaten_prey.to_csv("Data/most_eaten_prey.csv", index=False)

   taxon_id                     common_name scientific_name  predatorCount
0   47157.0           Butterflies and Moths     Lepidoptera             38
1   47178.0               Ray-finned Fishes  Actinopterygii             21
2  585956.0                 Common Wild Fig    Ficus burkei             21
3  205898.0                 Yellow Justicia  Justicia flava             15
4  184884.0  Winged and Once-winged Insects       Pterygota             15


In [91]:
heavily_eaten_nonpredators = gds.run_cypher("""
MATCH (prey:Taxa)-[:eaten_by]->(pred:Taxa)          // prey eaten by predators
WHERE NOT EXISTS { MATCH (:Taxa)-[:eaten_by]->(prey) } // prey does not eat others (no incoming prey->prey edges)
WITH prey, count(DISTINCT pred) AS predatorCount
ORDER BY predatorCount DESC
RETURN prey.taxon_id        AS taxon_id,
       prey.common_name     AS common_name,
       prey.scientific_name AS scientific_name,
       predatorCount
LIMIT 5
""")
print(heavily_eaten_nonpredators)
heavily_eaten_nonpredators.to_csv("Data/heavily_eaten_nonpredators.csv", index=False)

   taxon_id                     common_name scientific_name  predatorCount
0   47157.0           Butterflies and Moths     Lepidoptera             38
1   47178.0               Ray-finned Fishes  Actinopterygii             21
2  585956.0                 Common Wild Fig    Ficus burkei             21
3  205898.0                 Yellow Justicia  Justicia flava             15
4  184884.0  Winged and Once-winged Insects       Pterygota             15


### Critical Species 
- Species with high betweeness centrality 
- We are only looking for species with some betweeness
- In total_connections - we are adding the predator_count and prey_count

In [ ]:
#Key species based on high betweenness centrality
key_species_query = """
MATCH (n:Taxa)
WHERE n.betweenness > 0
WITH n,
     size([(n)<-[:eaten_by]-(prey) | prey]) as prey_count,
     size([(n)-[:eaten_by]->(pred) | pred]) as predator_count
RETURN 
    n.taxon_id as taxon_id,
    n.common_name as common_name,
    n.scientific_name as scientific_name,
    n.betweenness as betweenness,
    n.degree as degree,
    prey_count,
    predator_count,
    prey_count + predator_count as total_connections
ORDER BY n.betweenness DESC
LIMIT 20
"""


In [ ]:
keystone_df = gds.run_cypher(key_species_query)
print("\n" + "="*80)
print("KEYSTONE SPECIES (Critical Network Connectors)")
print("="*80)
print(keystone_df)


KEYSTONE SPECIES (Critical Network Connectors)
     taxon_id                       common_name  \
0     47219.0                 Western Honey Bee   
1     61355.0              Western Yellowjacket   
2    199400.0                         Siam weed   
3    676794.0              Common Dotted Border   
4     46017.0             Eastern Gray Squirrel   
5     12727.0                    American Robin   
6     46260.0             American Red Squirrel   
7     61495.0                  Eastern Pondhawk   
8    625887.0  Southern Brown-hooded Kingfisher   
9    483731.0                              None   
10  1482308.0                     Dancing Amber   
11    13858.0                     House Sparrow   
12   119994.0              Eastern Yellowjacket   
13   118552.0                      Domestic Cat   
14   343067.0                         Fever Tea   
15    53905.0                   European Mantis   
16    94105.0                      European Eel   
17    68492.0              Tropica

### Generalist vs Specialist Predators
- We are trying to find whether predators eat many different things or just a few (specialist).
- prey_diversity - This is basically counting how many different prey species are present



In [ ]:
generalist_predators_query = """
MATCH (pred:Taxa)<-[:eaten_by]-(prey)
WITH pred, count(DISTINCT prey) as prey_diversity
WHERE prey_diversity > 0
RETURN 
    pred.taxon_id as taxon_id,
    pred.common_name as common_name,
    pred.scientific_name as scientific_name,
    prey_diversity,
    pred.degree as degree,
    CASE 
        WHEN prey_diversity >= 5 THEN 'Generalist'
        WHEN prey_diversity >= 3 THEN 'Moderate'
        ELSE 'Specialist'
    END as feeding_strategy
ORDER BY prey_diversity DESC
LIMIT 20
"""

In [ ]:
generalist_df = gds.run_cypher(generalist_predators_query)
print("\n" + "="*80)
print("GENERALIST PREDATORS (Diverse Diet)")
print("="*80)
print(generalist_df)


GENERALIST PREDATORS (Diverse Diet)
     taxon_id                       common_name  \
0      4956.0                  Great Blue Heron   
1     47219.0                 Western Honey Bee   
2    625887.0  Southern Brown-hooded Kingfisher   
3     81839.0                Tan Jumping Spider   
4    116999.0                            Osprey   
5    243970.0                      Olympic Gull   
6      4399.0              Glaucous-winged Gull   
7     55746.0             Goldenrod Crab Spider   
8     53809.0               Bold Jumping Spider   
9      4954.0                        Grey Heron   
10    12942.0                  Eastern Bluebird   
11   126889.0             Margined Calligrapher   
12   144455.0                       Great Egret   
13     5305.0                        Bald Eagle   
14  1454382.0          Double-crested Cormorant   
15     8021.0                     American Crow   
16    68901.0              Zebra Jumping Spider   
17    12727.0                    American Rob

### Most Vulnerable Prey
- Prey species that many different predators hunt

In [ ]:
vulnerable_prey_query = """
MATCH (prey:Taxa)-[:eaten_by]->(pred)
WITH prey, count(DISTINCT pred) as predator_diversity
WHERE predator_diversity > 0
RETURN 
    prey.taxon_id as taxon_id,
    prey.common_name as common_name,
    prey.scientific_name as scientific_name,
    predator_diversity,
    prey.betweenness as betweenness
ORDER BY predator_diversity DESC
LIMIT 20
"""

In [ ]:
vulnerable_df = gds.run_cypher(vulnerable_prey_query)
print("\n" + "="*80)
print("MOST VULNERABLE PREY (Multiple Predators)")
print("="*80)
print(vulnerable_df)


MOST VULNERABLE PREY (Multiple Predators)
    taxon_id                              common_name  \
0    47157.0                    Butterflies and Moths   
1    47178.0                        Ray-finned Fishes   
2   585956.0                          Common Wild Fig   
3   205898.0                          Yellow Justicia   
4   184884.0           Winged and Once-winged Insects   
5    47219.0                        Western Honey Bee   
6    47822.0                                    Flies   
7    47118.0                                  Spiders   
8    47124.0                                   dicots   
9    83435.0                       American persimmon   
10   47158.0                                  Insects   
11  600542.0                        Coastal Resintree   
12   47649.0                Short-horned Grasshoppers   
13   60132.0                         creeping thistle   
14   47604.0  sunflowers, daisies, asters, and allies   
15  559524.0                             Duik

### Position in the Food Chain
- How many steps is each species from the top predators

Trophic levels help understand energy flow. Only ~10% of energy transfers between levels, so longer chains are less efficient and more fragile.

In [ ]:
trophic_levels_query = """
MATCH (n:Taxa)
OPTIONAL MATCH path = (n)-[:eaten_by*]->(apex)
WHERE NOT (apex)-[:eaten_by]->()
WITH n, max(length(path)) as max_path_to_apex
RETURN 
    n.taxon_id as taxon_id,
    n.common_name as common_name,
    n.scientific_name as scientific_name,
    COALESCE(max_path_to_apex, 0) as trophic_level,
    n.degree as degree,
    CASE 
        WHEN max_path_to_apex IS NULL OR max_path_to_apex = 0 THEN 'Apex Predator'
        WHEN max_path_to_apex = 1 THEN 'Secondary Consumer'
        WHEN max_path_to_apex = 2 THEN 'Primary Consumer'
        WHEN max_path_to_apex >= 3 THEN 'Producer/Base'
        ELSE 'Unknown'
    END as ecological_role
ORDER BY max_path_to_apex DESC
LIMIT 30
"""

In [ ]:

trophic_df = gds.run_cypher(trophic_levels_query)
print("\n" + "="*80)
print("TROPHIC LEVELS (Position in Food Chain)")
print("="*80)
print(trophic_df)


TROPHIC LEVELS (Position in Food Chain)
     taxon_id                 common_name               scientific_name  \
0    127147.0                        None            Adejeania vexatrix   
1    458903.0                        None               Aedes melanimon   
2    320034.0         Ocellate Gall Midge          Acericecis ocellaris   
3     52506.0     Two-spotted Lady Beetle             Adalia bipunctata   
4    201806.0               Pitted Beetle            Adesmia cancellata   
5    367917.0             Axehead Skipper               Acada biseriata   
6   1642428.0  Notched Shield Grasshopper         Abisares viridipennis   
7    346813.0   box elder pouch gall mite                Aceria negundi   
8    145095.0                  Javan Myna        Acridotheres javanicus   
9    915224.0                        None           Aculus minutissimus   
10   532817.0                        None       Adelphocoris seticornis   
11   116804.0               Sedge Warbler    Acrocephalus s

### Omnivorus
- Species that both eat and are eaten
- We are looking for the ones which are prey_count and predator_count more than 0
- We are using min_count = we want species balanced between eating and being eaten

In [ ]:
omnivores_query = """
MATCH (n:Taxa)
WITH n,
     size([(n)<-[:eaten_by]-(prey) | prey]) as prey_count,
     size([(n)-[:eaten_by]->(pred) | pred]) as predator_count
WHERE prey_count > 0 AND predator_count > 0
WITH n, prey_count, predator_count,
     CASE WHEN prey_count < predator_count THEN prey_count ELSE predator_count END as min_count
RETURN 
    n.taxon_id as taxon_id,
    n.common_name as common_name,
    n.scientific_name as scientific_name,
    prey_count,
    predator_count,
    n.betweenness as betweenness,
    abs(prey_count - predator_count) as balance_score
ORDER BY min_count DESC, n.betweenness DESC
LIMIT 20
"""

In [ ]:
omnivores_df = gds.run_cypher(omnivores_query)
print("\n" + "="*80)
print("OMNIVORES / MIDDLE TROPHIC LEVEL SPECIES")
print("="*80)
print(omnivores_df)


OMNIVORES / MIDDLE TROPHIC LEVEL SPECIES
    taxon_id                       common_name  \
0    47219.0                 Western Honey Bee   
1    46017.0             Eastern Gray Squirrel   
2   199400.0                         Siam weed   
3    61495.0                  Eastern Pondhawk   
4    13858.0                     House Sparrow   
5   119994.0              Eastern Yellowjacket   
6    68492.0              Tropical House Gecko   
7   121322.0                 African Honey Bee   
8    61355.0              Western Yellowjacket   
9    46260.0             American Red Squirrel   
10  625887.0  Southern Brown-hooded Kingfisher   
11  557401.0                     Southern Lion   
12   48484.0                 Asian Lady Beetle   
13  676794.0              Common Dotted Border   
14   12727.0                    American Robin   
15  483731.0                              None   
16  118552.0                      Domestic Cat   
17   53905.0                   European Mantis   
18   941

### Isolated Species
- Species with no recorded feeding relationship
- These species would have no degree

What could be the reasons?
- Could be due to data gap
- They are decomposers which were not captured
etc


In [ ]:
isolated_species_query = """
MATCH (n:Taxa)
WHERE n.degree = 0
RETURN 
    n.taxon_id as taxon_id,
    n.common_name as common_name,
    n.scientific_name as scientific_name
"""

In [ ]:
isolated_df = gds.run_cypher(isolated_species_query)
print("\n" + "="*80)
print("ISOLATED SPECIES (No Recorded Feeding Relationships)")
print("="*80)
print(f"Total isolated species: {len(isolated_df)}")
print(isolated_df.head(10))


ISOLATED SPECIES (No Recorded Feeding Relationships)
Total isolated species: 1590
    taxon_id                 common_name          scientific_name
0   366899.0            Paradise Skipper        Abantis paradisea
1  1642428.0  Notched Shield Grasshopper    Abisares viridipennis
2   122099.0                  rosary pea        Abrus precatorius
3   367917.0             Axehead Skipper          Acada biseriata
4   646406.0       Mispel Leaf Gall Mite          Acalitus mallyi
5   281562.0                  Copperleaf      Acalypha wilkesiana
6   425082.0                        None      Acanthaspis obscura
7    51709.0       Starbellied Orbweaver    Acanthepeira stellata
8   145300.0                     Redpoll         Acanthis flammea
9    84994.0       Giant leaf-footed bug  Acanthocephala declivis


In [43]:
iso="""
MATCH (t:Taxa)
WHERE NOT EXISTS { (t)-[:eaten_by]->() }
  AND NOT EXISTS { ()-[:eaten_by]->(t) }
RETURN t.taxon_id, t.common_name, t.scientific_name
ORDER BY t.taxon_id;
"""
iso_df=gds.run_cypher(iso)
print("\nIsolated Species (No Predators or Prey):")


Isolated Species (No Predators or Prey):


In [44]:
isolated = gds.run_cypher("""
MATCH (t:Taxa)
WHERE EXISTS { MATCH (t)-[:eaten_by]->(t) }
  AND NOT EXISTS { MATCH (t)-[:eaten_by]->(other) WHERE other <> t }
  AND NOT EXISTS { MATCH (other)-[:eaten_by]->(t) WHERE other <> t }
RETURN t.taxon_id AS taxon_id, t.common_name AS common_name, t.scientific_name AS scientific_name
ORDER BY taxon_id
""")
print(isolated)


   taxon_id                         common_name          scientific_name
0    2599.0                   Common Kingfisher            Alcedo atthis
1   43112.0                        Brush Rabbit      Sylvilagus bachmani
2   52740.0         Northern highbush blueberry     Vaccinium corymbosum
3  245656.0                     Napoleon Spider          Synema globosum
4  347886.0              Adanson's House Jumper        Hasarius adansoni
5  490718.0  Highveld Lesser-Thicktail Scorpion  Uroplectes triangulifer


In [49]:
output_path = "Data/isolated.xlsx"  
isolated_df.to_excel(output_path, index=False)

print(f"Saved {len(isolated_df)} cycles to {output_path}")

Saved 1590 cycles to Data/isolated.xlsx


### Getting mutual connections where A eats B and B eats A

In [50]:
mutual_connections_query = """
MATCH (a:Taxa)-[:eaten_by]->(b:Taxa)
MATCH (b)-[:eaten_by]->(a)
WHERE id(a) < id(b)
RETURN 
    a.taxon_id as species_a_taxon_id,
    a.common_name as species_a_common_name,
    a.scientific_name as species_a_scientific_name,
    a.iconic_taxon_name as species_a_iconic_taxon,
    b.taxon_id as species_b_taxon_id,
    b.common_name as species_b_common_name,
    b.scientific_name as species_b_scientific_name,
    b.iconic_taxon_name as species_b_iconic_taxon
ORDER BY a.scientific_name, b.scientific_name
"""

In [51]:
mutual_df = gds.run_cypher(mutual_connections_query)

In [52]:
# Save to CSV
output_file = "Data/mutual_connections.csv"
mutual_df.to_csv(output_file, index=False)

In [53]:
print(f"\n✓ Found {len(mutual_df)} mutual predation pairs")
print(f"✓ Saved to: {output_file}")

if len(mutual_df) > 0:
    print(f"\nFirst few rows:")
    print(mutual_df.head(10))
else:
    print("\nNo mutual connections found (A eats B and B eats A)")


✓ Found 16 mutual predation pairs
✓ Saved to: Data/mutual_connections.csv

First few rows:
   species_a_taxon_id              species_a_common_name  \
0            132429.0                      Garden Locust   
1             94105.0                       European Eel   
2             94105.0                       European Eel   
3            244845.0            California Bee Assassin   
4             62292.0                     showy milkweed   
5            208507.0                 African Babul Blue   
6            339996.0               African Common White   
7           1020324.0  Colorado Black-notched Bumble Bee   
8            199400.0                          Siam weed   
9            199400.0                          Siam weed   

  species_a_scientific_name species_a_iconic_taxon  species_b_taxon_id  \
0    Acanthacris ruficornis                Insecta            625887.0   
1         Anguilla anguilla         Actinopterygii              4943.0   
2         Anguilla anguil

In [83]:
lep_counts = gds.run_cypher("""
MATCH (t:Taxa)
WHERE t.scientific_name = 'Lepidoptera' OR t.common_name = 'Lepidoptera'
OPTIONAL MATCH (t)-[:eaten_by]->(prey:Taxa)
OPTIONAL MATCH (pred:Taxa)-[:eaten_by]->(t)
WITH t,
     collect(DISTINCT prey) AS preyList,
     collect(DISTINCT pred) AS predatorList
RETURN t.taxon_id AS taxon_id,
       t.common_name AS common_name,
       t.scientific_name AS scientific_name,
       size(preyList) AS prey_count,
       size(predatorList) AS predator_count
""")
print(lep_counts)



   taxon_id            common_name scientific_name  prey_count  predator_count
0   47157.0  Butterflies and Moths     Lepidoptera          38               0
